<a href="https://colab.research.google.com/github/ZeroLeon/working_space/blob/master/TaobaoShoes_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 淘宝天猫平台运动鞋数据预处理

Import packages and load google.drive

In [6]:
import os 
from pathlib import Path
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)
base_dir = '/content/gdrive/My Drive/work_space'
path = Path(base_dir +'/data/')
path.mkdir(parents=True,exist_ok=True)
os.chdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install plotly==4.0.0

In [0]:
from fastai.text import *
from fastai.callbacks import *
import plotly.express as px

## Data Cleaning

In [3]:
%ls

淘宝天猫运动鞋20190726.xlsx


In [0]:
!mv 淘宝天猫运动鞋20190726.xlsx taobao_shoes.xlsx

In [5]:
%ls

taobao_shoes.xlsx


In [0]:
df = pd.read_excel('taobao_shoes.xlsx',header=1)

In [18]:
df.head()

,Id,UserId,PPath,ItemId,Type,OriginUrl,OriginUrlMd5,URL,CategoryId,Category,Keyword,Title,Image,Content,SellerId,Seller,SellerLocation,Price,OriginPrice,UnitPrice,TradeNum,Comments,GoodComment,GoodCommentRate,NormalComment,NormalCommentRate,Rate,RateSum,ShangHaiExpress,Collection,Place,Provice,CrawlTime,Sha,UrlNo,BrandName,RateStatus,BadCommentRate,BadComment,PicComment,AverageScore,HasStock,StockNum,SkuId,ShelfTime,DetailStatus,ShopUrl,ShopName,ProductionPlace,ShopId,AreaId,AreaName,service_promise,payment,stock,specification,promotions_infor,navigation,SkuMap,DetailsImgs,ImgStatus,GetPrice,goodsPostageInfo,arrivalTime,PromotionList
0,2696064,890482188,NaN,563037197945,2,NaN,NaN,http://mclick.simba.taobao.com/cc_im?p=%D4%CB%...,NaN,NaN,运动鞋 耐克,Nike 耐克官方 NIKE TESSEN 女子运动鞋休闲鞋 AA2172-tmall.com天猫,http://g.search1.alicdn.com/img/i1/31591003/O1...,"产品名称：Nike/耐克 AA2172,品牌:Nike/耐克,鞋码:35.53636.537...",890482188,nike官方旗舰店,上海,389.0,549.0,549.0,7272,14123,NaN,NaN,NaN,NaN,0,NaN,0,298133,NaN,NaN,NaN,NaN,1359,Nike/耐克,0,NaN,NaN,NaN,4.8,NaN,NaN,NaN,NaN,1,https://nike.tmall.com,NIKE官方旗舰店,NaN,71955116,NaN,NaN,正品保证 极速退款 赠运费险 七天无理由退换,信用卡 快捷支付 蚂蚁花呗 余额宝,0,NaN,NaN,NaN,"41;100白/白黑;3568394784509;549.00;62,39;100白/白黑;...",NaN,NaN,NaN,NaN,NaN,NaN
1,2696065,612456912,NaN,594227604532,2,NaN,NaN,https://detail.tmall.com/item.htm?id=594227604...,50012036.0,NaN,运动鞋 耐克,Nike耐克男鞋2019新款正品REACT ELEMENT 55运动跑步鞋BQ6166-00...,http://g.search2.alicdn.com/img/bao/uploaded/i...,"品牌:Nike/耐克,功能:耐磨轻便,闭合方式:系带,鞋码:394040.5414242.5...",612456912,炫动运动户外专营店,浙江 杭州,769.0,769.0,769.0,2,1,NaN,NaN,NaN,NaN,0,NaN,0,36,NaN,NaN,NaN,NaN,1359,Nike/耐克,0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,1,https://xdydhw.tmall.com,炫动运动户外专营店,NaN,63496077,NaN,NaN,正品保证 极速退款 七天无理由退换,信用卡 快捷支付 蚂蚁花呗 余额宝,0,NaN,NaN,NaN,42;BQ6166-009/加入购物车优先发货;4115443508745;769.00;7...,NaN,NaN,NaN,NaN,NaN,NaN
2,2696067,1993730769,NaN,520311488613,2,NaN,NaN,https://detail.tmall.com/item.htm?id=520311488...,50012043.0,NaN,运动鞋 Adidas,Adidas阿迪达斯男鞋2019夏季新款运动鞋鞋子透气休闲鞋小白板鞋-tmall.com天猫,http://g.search1.alicdn.com/img/bao/uploaded/i...,"产品名称：Adidas/阿迪达斯 2015Q3N...,品牌:Adidas/阿迪达斯,功能:...",1993730769,幸运叶子官方旗舰店,江苏省 连云港市,279.0,279.0,279.0,3018,7265,NaN,NaN,NaN,NaN,0,NaN,0,33115,NaN,NaN,NaN,NaN,1359,Adidas/阿迪达斯,0,NaN,NaN,NaN,4.6,NaN,NaN,NaN,NaN,1,https://xingyunyeziguanfang.tmall.com,幸运叶子官方旗舰店,NaN,108837944,NaN,NaN,正品保证 极速退款 七天无理由退换,信用卡 快捷支付 蚂蚁花呗 余额宝,0,NaN,NaN,NaN,"39;F34252;4167515129307;379.00;0,44;EE9672;416...",NaN,NaN,NaN,NaN,NaN,NaN
3,2696068,890482188,NaN,597080923839,2,NaN,NaN,https://detail.tmall.com/item.htm?id=597080923...,50012036.0,NaN,运动鞋 耐克,Nike 耐克官方NIKE ZOOM PEGASUS TURBO 2男子跑步鞋AT2863-...,http://g.search1.alicdn.com/img/bao/uploaded/i...,"品牌:Nike/耐克,功能:减震,鞋码:38.5394040.5414242.5434444...",890482188,nike官方旗舰店,上海 上海,1399.0,1399.0,1399.0,323,63,NaN,NaN,NaN,NaN,0,NaN,0,2949,NaN,NaN,NaN,NaN,1359,Nike/耐克,0,NaN,NaN,NaN,4.9,NaN,NaN,NaN,NaN,1,https://nike.tmall.com,NIKE官方旗舰店,NaN,71955116,NaN,NaN,正品保证 极速退款 赠运费险 七天无理由退换,信用卡 快捷支付 蚂蚁花呗 余额宝,0,NaN,NaN,NaN,"46;001黑/白色/枪灰色/大气灰;4332042602210;1399.00;12,40...",NaN,NaN,NaN,NaN,NaN,NaN
4,2696069,890482188,NaN,594003882472,2,NaN,NaN,https://detail.tmall.com/item.htm?id=594003882...,50012043.0,NaN,运动鞋 Nike,Nike 耐克官方 NIKE M2K TEKNO 男子运动鞋休闲鞋 CI5749-tmall...,http://g.search2.alicdn.com/img/bao/uploaded/i...,"品牌:Nike/耐克,鞋码:38.5394040.5414242.5434444.54546...",890482188,nike官方旗舰店,上海 上海,799.0,799.0,799.0,1258,310,NaN,NaN,NaN,NaN,0,NaN,0,11043,NaN,NaN,NaN,NaN,1359,Nike/耐克,0,NaN,NaN,NaN,4.7,NaN,NaN,NaN,NaN,1,https://nike.tmall.com,NIKE官方旗舰店,NaN,71955116,NaN,NaN,正品保证 极速退款 赠运费险 七天无理由退换,信用卡 快捷支付 蚂蚁花呗 余额宝,0,NaN,NaN,NaN,"44;888荷兰橙/荷兰橙/荷兰橙;4274793314987;799.00;172,38....",NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#List all columns cotains only NaN values
drop_list = df.columns[df.isna().all()].tolist()

In [32]:
len(drop_list)

21

In [0]:
df.drop(columns = drop_list,inplace = True )

In [30]:
len(df.columns)

44

In [0]:
df_des = df.isna().describe()

In [40]:
df_des.head()

,Id,UserId,ItemId,Type,URL,CategoryId,Keyword,Title,Image,Content,SellerId,Seller,SellerLocation,Price,OriginPrice,UnitPrice,TradeNum,Comments,GoodComment,GoodCommentRate,NormalComment,NormalCommentRate,Rate,ShangHaiExpress,Collection,Place,Provice,UrlNo,BrandName,RateStatus,BadCommentRate,BadComment,PicComment,AverageScore,StockNum,DetailStatus,ShopUrl,ShopName,ShopId,service_promise,payment,stock,promotions_infor,SkuMap
count,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097,39097
unique,1,1,1,1,1,2,1,1,1,2,1,1,2,1,1,1,1,1,2,2,2,2,1,1,1,2,2,1,2,1,2,2,2,1,2,1,1,2,1,2,2,1,2,1
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,False,False,False,True,True,False,False,False,True,True,True,False,True,False,False,False,False,False,False,False,True,False
freq,39097,39097,39097,39097,39097,38671,39097,39097,39097,39089,39097,39097,26247,39097,39097,39097,39097,39097,21456,21456,21456,21456,39097,39097,39097,39019,39062,39097,39082,39097,21456,21456,21456,39097,39093,39097,39097,39068,39097,21369,21372,39097,39094,39097


In [0]:
#List all cols with most values as Nan
drop_list2 = df.columns[(df_des.loc['top'] == True) & (df_des.loc['freq'] > 30000)].tolist()

In [0]:
df.drop(columns = drop_list2,inplace = True )

In [49]:
len(df.columns)

40

##  Trim away the columns highly correlated or with constant value 


*   **GoodComment** is highly correlated with **Comments**
*   **OriginPrice** is highly correlated with **Price**
*   SellerId is highly correlated with UserId
*   **UnitPrice** is highly correlated with **OriginPrice**
*   DetailStatus has constant value 1
*   Rate has constant value 0
*   RateStatus has constant value 0 
*   ShangHaiExpress has constant value 0
*   UrlNo has constant value 1359
*   stock has constant value 0





价格的三个字段高度相关，总评中好评占绝大多数，也就是说差评与打折价可以作为异常值看待，留给  Anomaly Detection 的模型处理，在这里先去除多余字段只保留一个。

In [70]:
df.GoodComment.value_counts().count()

1298

In [76]:
df[['Comments','GoodComment']].sample(10)

,Comments,GoodComment
12632,418,NaN
12835,0,0.0
12832,7,6.0
8470,569,563.0
27942,24,NaN
38820,2,NaN
26287,28,NaN
24128,17,NaN
19623,699,NaN
11348,9,8.0


In [0]:
remove_cols = ['GoodComment','OriginPrice','UnitPrice','UserId','DetailStatus','Rate','RateStatus','ShangHaiExpress','UrlNo','stock']

In [0]:
df.to_csv('taobao_shoes_nona.csv',index=None)

In [0]:
#Remove extra columns
df.drop(columns = remove_cols,inplace=True)

In [83]:
df.columns.shape

(30,)

In [94]:
df.dtypes.value_counts()

object     13
float64     9
int64       8
dtype: int64

In [95]:
df.head().T

,0,1,2,3,4
Id,2696064,2696065,2696067,2696068,2696069
ItemId,563037197945,594227604532,520311488613,597080923839,594003882472
Type,2,2,2,2,2
URL,http://mclick.simba.taobao.com/cc_im?p=%D4%CB%...,https://detail.tmall.com/item.htm?id=594227604...,https://detail.tmall.com/item.htm?id=520311488...,https://detail.tmall.com/item.htm?id=597080923...,https://detail.tmall.com/item.htm?id=594003882...
CategoryId,NaN,5.0012e+07,5.0012e+07,5.0012e+07,5.0012e+07
Keyword,运动鞋 耐克,运动鞋 耐克,运动鞋 Adidas,运动鞋 耐克,运动鞋 Nike
Title,Nike 耐克官方 NIKE TESSEN 女子运动鞋休闲鞋 AA2172-tmall.com天猫,Nike耐克男鞋2019新款正品REACT ELEMENT 55运动跑步鞋BQ6166-00...,Adidas阿迪达斯男鞋2019夏季新款运动鞋鞋子透气休闲鞋小白板鞋-tmall.com天猫,Nike 耐克官方NIKE ZOOM PEGASUS TURBO 2男子跑步鞋AT2863-...,Nike 耐克官方 NIKE M2K TEKNO 男子运动鞋休闲鞋 CI5749-tmall...
Image,http://g.search1.alicdn.com/img/i1/31591003/O1...,http://g.search2.alicdn.com/img/bao/uploaded/i...,http://g.search1.alicdn.com/img/bao/uploaded/i...,http://g.search1.alicdn.com/img/bao/uploaded/i...,http://g.search2.alicdn.com/img/bao/uploaded/i...
Content,"产品名称：Nike/耐克 AA2172,品牌:Nike/耐克,鞋码:35.53636.537...","品牌:Nike/耐克,功能:耐磨轻便,闭合方式:系带,鞋码:394040.5414242.5...","产品名称：Adidas/阿迪达斯 2015Q3N...,品牌:Adidas/阿迪达斯,功能:...","品牌:Nike/耐克,功能:减震,鞋码:38.5394040.5414242.5434444...","品牌:Nike/耐克,鞋码:38.5394040.5414242.5434444.54546..."
SellerId,890482188,612456912,1993730769,890482188,890482188


In [0]:
df.to_csv('taobao_shoes_30col.csv',index=None)

In [0]:
# Make categorical data get correct datatypes
df['CategoryId'] = df.CategoryId.astype('category')
df['payment'] = df.payment.astype('category')
df['service_promise'] = df.service_promise.astype('category')

## Focus on some features 

In [0]:
df = pd.read_csv('taobao_shoes_30col.csv')

In [0]:
attention_list = ['SellerLocation','Price','TradeNum','Comments','Collection','AverageScore','ShopName','BrandName','Keyword']

In [0]:
df.head().T

In [0]:
#focused data file 
df_f = df[attention_list].copy()

In [19]:
df_f.head()

,SellerLocation,Price,TradeNum,Comments,Collection,AverageScore,ShopName,BrandName,Keyword
0,上海,389.0,7272,14123,298133,4.8,NIKE官方旗舰店,Nike/耐克,运动鞋 耐克
1,浙江 杭州,769.0,2,1,36,5.0,炫动运动户外专营店,Nike/耐克,运动鞋 耐克
2,江苏省 连云港市,279.0,3018,7265,33115,4.6,幸运叶子官方旗舰店,Adidas/阿迪达斯,运动鞋 Adidas
3,上海 上海,1399.0,323,63,2949,4.9,NIKE官方旗舰店,Nike/耐克,运动鞋 耐克
4,上海 上海,799.0,1258,310,11043,4.7,NIKE官方旗舰店,Nike/耐克,运动鞋 Nike


In [20]:
df_f.dtypes

SellerLocation     object
Price             float64
TradeNum            int64
Comments            int64
Collection          int64
AverageScore      float64
ShopName           object
BrandName          object
Keyword            object
dtype: object

In [21]:
df_f.isna().any()

SellerLocation     True
Price             False
TradeNum          False
Comments          False
Collection        False
AverageScore      False
ShopName           True
BrandName          True
Keyword           False
dtype: bool

In [22]:
df_f.BrandName.isna().value_counts()

False    39082
True        15
Name: BrandName, dtype: int64

In [23]:
df_f.ShopName.isna().value_counts()

False    39068
True        29
Name: ShopName, dtype: int64

In [24]:
df_f.SellerLocation.isna().value_counts()

False    26247
True     12850
Name: SellerLocation, dtype: int64

In [0]:
#Remove NaN values of 2 subsets
df_f.dropna(subset=['ShopName','BrandName'],inplace=True)

## Visualization 

In [65]:
fig = px.histogram(df_f,x='Price',nbins = 500,template='plotly_dark')
fig.show()

Cut a slice of 0-2000, to see the distribution of each features 

In [83]:
fig = px.histogram(df_f[df_f.Price<2001],x='Price',
                   y='TradeNum',barmode='group',
                   nbins=60,template='plotly_dark',
                   title = 'Trade Number Variation by Price',
                  orientation="v"
                  )
fig.show()

In [37]:
df_count = df_f.groupby(pd.cut(df_viz['Price'], np.arange(0, 2000, 200))).count().copy()
df_count

,SellerLocation,Price,TradeNum,Comments,Collection,AverageScore,ShopName,BrandName,Keyword
Price,,,,,,,,,
"(0, 200]",9510,15236,15236,15236,15236,15236,15236,15236,15236
"(200, 400]",7471,11403,11403,11403,11403,11403,11403,11403,11403
"(400, 600]",4850,6495,6495,6495,6495,6495,6495,6495,6495
"(600, 800]",2365,3087,3087,3087,3087,3087,3087,3087,3087
"(800, 1000]",973,1289,1289,1289,1289,1289,1289,1289,1289
"(1000, 1200]",313,419,419,419,419,419,419,419,419
"(1200, 1400]",222,310,310,310,310,310,310,310,310
"(1400, 1600]",81,116,116,116,116,116,116,116,116
"(1600, 1800]",87,138,138,138,138,138,138,138,138


In [0]:
df_count.index.names = ['Price_range']

In [39]:
df_count.reset_index(inplace=True)
df_count.head()

,Price_range,SellerLocation,Price,TradeNum,Comments,Collection,AverageScore,ShopName,BrandName,Keyword
0,"(0, 200]",9510,15236,15236,15236,15236,15236,15236,15236,15236
1,"(200, 400]",7471,11403,11403,11403,11403,11403,11403,11403,11403
2,"(400, 600]",4850,6495,6495,6495,6495,6495,6495,6495,6495
3,"(600, 800]",2365,3087,3087,3087,3087,3087,3087,3087,3087
4,"(800, 1000]",973,1289,1289,1289,1289,1289,1289,1289,1289


In [0]:
df_count['Price_range'] =  df_count.Price_range.apply(str)

In [84]:
fig = px.line(df_count, x="Price_range",
              y="SellerLocation",
              template='plotly_dark',
             labels=dict(Price_range="Price Range", SellerLocation="Location"),
              title = 'Location Numbers by Price Range',
             
             )
fig.show()

In [85]:
df_f.head()

,SellerLocation,Price,TradeNum,Comments,Collection,AverageScore,ShopName,BrandName,Keyword
0,上海,389.0,7272,14123,298133,4.8,NIKE官方旗舰店,Nike/耐克,运动鞋 耐克
1,浙江 杭州,769.0,2,1,36,5.0,炫动运动户外专营店,Nike/耐克,运动鞋 耐克
2,江苏省 连云港市,279.0,3018,7265,33115,4.6,幸运叶子官方旗舰店,Adidas/阿迪达斯,运动鞋 Adidas
3,上海 上海,1399.0,323,63,2949,4.9,NIKE官方旗舰店,Nike/耐克,运动鞋 耐克
4,上海 上海,799.0,1258,310,11043,4.7,NIKE官方旗舰店,Nike/耐克,运动鞋 Nike


In [0]:
keword_list = df_f.Keyword.value_counts().index.tolist()[:10]

In [130]:
#Use a part of data for visualization
df_f_viz= df_f[(df_f.Comments<5000)&(df.Price<5000)&(df.Keyword.isin(keyword_list))&(df.SellerLocation.notnull())]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [131]:
df_f_viz.shape

(15153, 9)

In [132]:
fig = px.scatter(df_f_viz,
                 x= 'Comments',
                 y='TradeNum',color='Keyword',            
                 size='Price',
                 hover_name="SellerLocation",  size_max=60,
                 template='plotly_dark',
                 title='Variation of Deferent Features'
             )
fig.show()

*   左下角靠近原点的密集部分，是销量低评论少的商品，正常情况下面积大（价格高），这一部分如果有低价商品会有假货嫌疑，可以深入探究。
*   右上角可以看到一些散点，代表低价高销量的爆款商品，是受到仿冒的高危区。
*   右下角代表评论数多但销量低的商品，可能是商品换新之类的原因导致，有待查看。

初步分析可以看到原始商品数据集存在大量**数据倾斜**，在之后的模型训练分析阶段需要单独建立**异常值检测模型**，可能会有比较好的效果。

## To be continued